In [6]:
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Загрузка изображения ---
image = Image.open('birch.jpg').convert("RGB")

from ultralytics import YOLO

# Установка: pip install ultralytics

# Загрузка модели YOLOv8 segmentation
model = YOLO('yolov8x-seg.pt')  # или yolov8s-seg.pt, yolov8m-seg.pt, yolov8l-seg.pt

# Выполнение сегментации
results = model(image)

# Получение масок
masks = results[0].masks

if masks is not None and len(masks) > 0:
    # Берем маску с наибольшей площадью (крупный объект)
    mask_areas = [np.sum(mask.data.cpu().numpy()[0]) for mask in masks]
    largest_mask_idx = np.argmax(mask_areas)
    mask = masks.data[largest_mask_idx].cpu().numpy()

    # Применение маски
    image_np = np.array(image)
    masked_img = image_np.copy()
    masked_img[~mask.astype(bool)] = 255
    masked_pil = Image.fromarray(masked_img)

# --- 3. Классификация BioCLIP ---
# Установи: pip install transformers timm
import open_clip

# Загрузка модели BioCLIP и токенизатора через open_clip
model_clip, preprocess, tokenizer = open_clip.create_model_and_transforms('hf-hub:imageomics/bioclip')
device = "cuda" if torch.cuda.is_available() else "cpu"
model_clip = model_clip.to(device)
model_clip.eval()

# Список классов (замени на нужные породы)
classes = ["birch tree", "oak tree", "pine tree", "maple tree", "spruce tree"]

# Подготовка изображения
# Предполагается, что masked_pil - это ваше изображение PIL
image_tensor = preprocess(masked_pil).unsqueeze(0).to(device)

# Подготовка текстов
text_tokens = tokenizer(classes).to(device)

# Получение эмбеддингов и вычисление логитов
with torch.no_grad():
    image_features = model_clip.encode_image(image_tensor)
    text_features = model_clip.encode_text(text_tokens)
    
    # Нормализация эмбеддингов
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    
    # Вычисление логитов (сходства)
    logits_per_image = (100.0 * image_features @ text_features.T)
    probs = logits_per_image.softmax(dim=1).cpu().numpy()[0]

# --- 4. Вывод результата ---
for cls, p in zip(classes, probs):
    print(f"{cls:15s}: {p:.3f}")

print("\nPredicted:", classes[int(probs.argmax())])

# Визуализация
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Original")

plt.subplot(1, 2, 2)
plt.imshow(masked_pil)
plt.title("Masked Tree")
plt.show()



0: 640x480 (no detections), 967.5ms
Speed: 32.1ms preprocess, 967.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 480)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


KeyboardInterrupt: 